In [ ]:
import pandas as pd
import numpy as np
import feature_engine
import os
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import seaborn as sns
sns.set()
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from itertools import product

from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Spis treści
* [1. Exploratory Data Analysis](#eda)
* [2. SARIMA](#sarima)
* [3. Modele autoregresyjne](#auto_reg)
* [3.1. Podział zbioru na zbiór treningowy i testowy](#test_train_split)
* [3.2. XGboost, Random Forest - autoregresyjne](#auto_reg_ml)
* [4. Feature engineering](#FE_all)
* [4.1. Zamówienia klienta](#FE_cli)
* [4.2. Dane dot. realnych realizacji usługi](#FE_sales)
* [4.3. Dane kalendarzowe](#FE_calendar)
* [4.3. Dane pogodowe](#FE_pogodowe)
* [5. Trening modelu](#model_fin)
* [6. Zestawienie wyników prognoz](#comparison)
* [7. Pomysły na ulepszenie algorytmu ](#poprawa)

<a id="eda"></a>
# 1. Exploratory Data Analysis

In [ ]:
data = pd.read_csv("data/dimDates_Task.csv", sep=";", index_col=['dateId'], parse_dates=['dateId'])
data.head()

In [ ]:
# liczba nadań
volumes_df = pd.read_parquet('data/Posting_Volumes')
volumes_df['postingDateFk'] = pd.to_datetime(volumes_df['postingDateFk'], format='%Y%m%d')
volumes_df.set_index('postingDateFk', inplace=True)
volumes_df.sort_values(by='postingDateFk', ascending=False).head()

In [ ]:
volumes_df.Customer.value_counts()

In [ ]:
volumes_df.Product.value_counts()

In [ ]:
volumes_df.Volume.describe()

In [ ]:
volumes_df = volumes_df.query("postingDateFk >= '2023-01-01'")
volumes_df = volumes_df.query("Customer == 'X'")

In [ ]:
volumes_df_test = volumes_df.query("postingDateFk >= '2023-08-01'")
volumes_df_test = volumes_df_test[['Volume']].groupby('postingDateFk').sum()
volumes_df = volumes_df.query("postingDateFk < '2023-08-01'")

In [ ]:
orders_df = pd.read_excel('data/X_ClientORDERS.xlsx')
orders_df['DateId'] = pd.to_datetime(orders_df['DateId'], format='%Y%m%d')
orders_df.set_index('DateId', inplace=True)
orders_df.head()

In [ ]:
grouped_sum = volumes_df[['Volume']].groupby('postingDateFk').sum()
merged_df = grouped_sum.merge(orders_df, left_index=True, right_index=True, how='left')

In [ ]:
plt.figure(figsize=(17, 8))
plt.plot(merged_df.index, merged_df['Volume'], label='Volume Real')
plt.plot(merged_df.index, merged_df['Orders'], label='Orders Predicted')

plt.title('Szeregi czasowe dla predykcji i prawdziwej realizacji zamówień klienta X')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
grouped_sum = volumes_df.pivot_table(values='Volume', index='postingDateFk', columns='Product', aggfunc='sum')
merged_df = grouped_sum.merge(orders_df, left_index=True, right_index=True, how='left')

In [ ]:
merged_df

In [ ]:
plt.figure(figsize=(17, 8))
plt.plot(merged_df.index, merged_df['APM'], label='APM Real')
plt.plot(merged_df.index, merged_df['Orders'], label='Orders Predicted')
plt.plot(merged_df.index, merged_df['COURIER'], label='Courier Real')

plt.title('Szeregi czasowe dla predykcji i prawdziwej realizacji zamówień klienta X wg typu produktu')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

 - Klient przysyła swoją predykcję zamówień na kolejny miesiąc. Przesłanie swojej predykcji zamówień można potraktować jako zgłoszenie zapotrzebowania na usługę
 
 - Zapotrzebowanie zgłoszone przez klienta jest różne od obserwowanej później liczby realnych nadań
 
 - Na podstawie jego zadeklarowanego zapotrzebowania, historycznych realizacji, uwzględnienia dodatkowych czynników takich jak święta, pogoda oraz dodatkowe zmienne możemy dokładniej przewidzieć jakie rzeczywiście będzie zapotrzebowanie na usługi

In [ ]:
volumes_df = volumes_df[['Volume']].groupby('postingDateFk').sum()

In [ ]:
folder_path = 'data/'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.startswith('k_') and filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
pogoda_df = pd.concat(dataframes, ignore_index=True)
pogoda_df.head()

zmienna prognozowana -
liczba nadań w konkretnym dniu kolejnego tygodnia/miesiąca

In [ ]:
plt.figure(figsize=(17, 8))
plt.plot(volumes_df.Volume)
plt.title('Dzienna liczba nadań')
plt.ylabel('Liczba nadań')
plt.xlabel('Dzień nadania')
plt.grid(False)
plt.show()

In [ ]:
def moving_average_plot(series, window, scale=1.96):
    rolling_mean = series.rolling(window=window).mean()
    plt.figure(figsize=(17,8))
    plt.title('Średnia ruchoma\n dla okna danych = {} dni'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')      
    plt.plot(series[window:], label='Liczba nadań')
    plt.legend(loc='best')
    plt.grid(True)

Wykres średniej ruchomej w celu identyfikacji trendów w danych. Jak zmieniają się dzienne wartości liczby nadań w stosunku do ich średniej ruchomej

In [ ]:
moving_average_plot(volumes_df.Volume, 7)

Wysoka dzienna zmienność

Wygładzenie wykładnicze w czasie: alfa - jak ważne są poprzednie wartości danych

In [ ]:
def exponential_smoothing(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result
  
def exponential_smoothing_plt(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing")
    plt.grid(True);

exponential_smoothing_plt(volumes_df.Volume, [0.03, 0.1, 0.5, 0.8])

Im mniejsza wartość alpha, tym większy wpływ mają wcześniejsze obserwacje.
Model bardziej reaguje na najnowsze obserwacje

<a id="sarima"></a>
# 2. SARIMA

In [ ]:
def ts_analysis_plot(y, lags=None, figsize=(12, 7), syle='bmh'):
    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style='bmh'):
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series \n Dickey-Fuller test: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.001)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.001)
        plt.tight_layout()

Parametr p - zakładamy, że bieżąca wartość zależy od poprzednich wartości z pewnym opóźnieniem, identyfikujemy opóźnienie, po którym większość opóźnień nie jest znacząca

Parametr q - reprezentuje największe opóźnienie, po którym inne opóźnienia nie są znaczące na wykresie autokorelacji

In [ ]:
ts_analysis_plot(volumes_df.Volume, lags=14)

data_diff = volumes_df.Volume - volumes_df.Volume.shift(1)

#ts_analysis_plot(data_diff[1:], lags=7)

Oba szeregi są stacjonarne:

p-value jest mniejsze od poziomu istotności (a poziomie 0.05), zatem odrzucamy hipotezę zerową i wnioskujemy, że szereg czasowy jest stacjonarny

In [ ]:
ps = range(0, 2)
d = 1
qs =  range(0, 2)
Ps = range(0, 2)
D = 1
Qs = range(0, 2)
s = 2

parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

def SARIMA_param_optimalization(parameters_list, d, D, s):
    """
        Metoda zwaraca ramkę danych z wartością AIC i parametrami modelu
        
        parameters_list - lista parametrów (p, q, P, Q)
        d - integration order
        D - seasonal integration order
        s - length of season
    """
    
    results = []
    best_aic = float('inf')
    
    for param in tqdm_notebook(parameters_list):
        try: model = sm.tsa.statespace.SARIMAX(volumes_df.Volume, order=(param[0], d, param[1]),
                                               seasonal_order=(param[2], D, param[3], s), freq='D').fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        
        # Zapis najlepszego modelu
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
        
    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    # Posortowanie w kolejności rosnącej, im niższe AIC tym lepszy model
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

result_table = SARIMA_param_optimalization(parameters_list, d, D, s)

# Wybieramy model z najniższym AIC (Akaike Information Criteria)
p, q, P, Q = result_table.parameters[0]

best_model = sm.tsa.statespace.SARIMAX(volumes_df.Volume, order=(p, d, q),
                                       seasonal_order=(P, D, Q, s)).fit(disp=-1)

print(best_model.summary())

In [ ]:
volumes_df.index.max()

In [ ]:
forecast = best_model.predict(start='2023-08-01', end='2023-08-31')

In [ ]:
comparison = pd.DataFrame({'Prawdziwa': volumes_df_test.Volume.values,
                          'Przewidywana': forecast.values}, 
                          index = pd.date_range(start='2023-08-01', periods=31,))

plt.figure(figsize=(17, 8))
plt.plot(comparison.Prawdziwa)
plt.plot(comparison.Przewidywana)
plt.title('Przewidywana liczba nadań')
plt.ylabel('Liczba nadań')
plt.xlabel('Dzień')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
comparison['diff']= comparison['Prawdziwa'] - comparison['Przewidywana']

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mse_sarima = mean_squared_error(comparison['Prawdziwa'], comparison['Przewidywana'])
mape_sarima = mean_absolute_percentage_error(comparison['Prawdziwa'], comparison['Przewidywana'])
mae_sarima = mean_absolute_error(comparison['Prawdziwa'], comparison['Przewidywana'])

print("Metryka MSE:", mse_sarima)
print("Metryka MAPE:", mape_sarima)
print("Metryka MAE:", mae_sarima)

<a id="auto_reg"></a>
# 3. Modele autoregresyjne

<a id="test_train_split"></a>
# 3.1. Train / test split

In [ ]:
abt = pd.read_parquet('data/Posting_Volumes')
abt['postingDateFk'] = pd.to_datetime(abt['postingDateFk'], format='%Y%m%d')
abt.set_index('postingDateFk', inplace=True)
abt = abt.query("postingDateFk >= '2022-01-01'")
abt = abt.query("Customer == 'X'")
abt = abt[['Volume']].groupby('postingDateFk').sum()
abt = abt.asfreq('D')
abt = abt['Volume'].dropna()
end_train = '2023-06-30 00:00:00'

In [ ]:
abt.index.max()

In [ ]:
abt.index.min()

In [ ]:
print(
    f"Train dates : {abt.index.min()} --- {abt.loc[:end_train].index.max()}  " 
    f"(n={len(abt.loc[:end_train])})")
print(
    f"Test dates  : {abt.loc[end_train:].index.min()} --- {abt.index.max()}  "
    f"(n={len(abt.loc[end_train:])})")

fig, ax = plt.subplots(figsize=(11, 3))

abt.plot(ax=ax)
ax.axvline(x=end_train, color='r', linestyle='--', label='Train/Test Split')
ax.legend()
ax.set_title('Train / test split')

plt.show()

<a id="auto_reg_ml"></a>
# 3.2. XGboost, Random Forest

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

steps = len(abt.loc[end_train:])

forecaster_rf = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=362),
                    lags      = 30
                )
forecaster_gb = ForecasterAutoreg(
                    regressor = XGBRegressor(random_state=362),
                    lags      = 30
                )

forecaster_rf.fit(abt.loc[:end_train])
forecaster_gb.fit(abt.loc[:end_train])

predictions_rf = forecaster_rf.predict(steps=steps)
predictions_gb = forecaster_gb.predict(steps=steps)

In [ ]:
error_rf = mean_absolute_error(abt.loc[end_train:], predictions_rf)
error_gb = mean_absolute_error(abt.loc[end_train:], predictions_gb)
print(f"Error (MAE) Random Forest: {error_rf:.2f}")
print(f"Error (MAE) Gradient Boosting: {error_gb:.2f}")

fig, ax = plt.subplots(figsize=(9, 6), sharex=True, sharey=True)
abt.loc[:end_train].plot(ax=ax, label='train')
abt.loc[end_train:].plot(ax=ax, label='test')
predictions_rf.plot(ax=ax, label='Random Forest')
predictions_gb.plot(ax=ax, label='Gradient Boosting')
ax.set_title(f'Modele autoregresyjne - predykcja')
ax.set_xlabel('')
ax.legend();

In [ ]:
mse_xgb_autoreg = mean_squared_error(abt.loc[end_train:], predictions_gb)
mape_xgb_autoreg = mean_absolute_percentage_error(abt.loc[end_train:], predictions_gb)
mae_xgb_autoreg = mean_absolute_error(abt.loc[end_train:], predictions_gb)

print("Metryka MSE:", mse_xgb_autoreg)
print("Metryka MAPE:", mape_xgb_autoreg)
print("Metryka MAE:", mae_xgb_autoreg)

In [ ]:
mse_rf_autoreg = mean_squared_error(abt.loc[end_train:], predictions_rf)
mape_rf_autoreg = mean_absolute_percentage_error(abt.loc[end_train:], predictions_rf)
mae_rf_autoreg = mean_absolute_error(abt.loc[end_train:], predictions_rf)

print("Metryka MSE:", mse_rf_autoreg)
print("Metryka MAPE:", mape_rf_autoreg)
print("Metryka MAE:", mae_rf_autoreg)

<a id="FE_all"></a>
# 4. Feature engineering

<a id="FE_cli"></a>
# 4.1 Dane klientowskie - Features

In [ ]:
orders_df = pd.read_excel('data/X_ClientORDERS.xlsx')
orders_df['DateId'] = pd.to_datetime(orders_df['DateId'], format='%Y%m%d')
orders_df.set_index('DateId', inplace=True)

In [ ]:
def lag_features(data, column, lag_steps=1):
    for i in range(1, lag_steps + 1):
        data[f'{column}_lag_{i}'] = data[column].shift(i)
    return data

In [ ]:
def rolling_agg(data, column, agg_func, window_size=2):
    if agg_func not in ['mean', 'sum', 'max', 'min']:
        raise ValueError("Value error")
    # -7 jeśli dla tychodniowych predykcji
    agg_result = getattr(data[column].shift(-7).rolling(window=window_size), agg_func)()
    data[f'{column}_{agg_func}_{window_size}d'] = agg_result
    return data

In [ ]:
def trend_features(data, column, lag_steps=2):
    data[f'{column}_trend_lag_{lag_steps}'] = (data[f'{column}_lag_{lag_steps-1}'] - data[f'{column}_lag_{lag_steps}']) / data[f'{column}_lag_{lag_steps}']
    return data

In [ ]:
orders_df = lag_features(orders_df, 'Orders', lag_steps=15)

orders_df = rolling_agg(orders_df,'Orders', 'mean', window_size= 7)
orders_df = rolling_agg(orders_df,'Orders', 'sum', window_size= 7)
orders_df = rolling_agg(orders_df,'Orders', 'max', window_size= 7)
orders_df = rolling_agg(orders_df,'Orders', 'min', window_size= 7)

orders_df = rolling_agg(orders_df,'Orders', 'mean', window_size= 10)
orders_df = rolling_agg(orders_df,'Orders', 'sum', window_size= 10)
orders_df = rolling_agg(orders_df,'Orders', 'max', window_size= 10)
orders_df = rolling_agg(orders_df,'Orders', 'min', window_size= 10)

orders_df = trend_features(orders_df, 'Orders', lag_steps=8)
orders_df = trend_features(orders_df, 'Orders', lag_steps=9)

# orders_df['Day'] = orders_df.index.day
# orders_df['Month'] = orders_df.index.month

In [ ]:
orders_df.head()

<a id="FE_sales"></a>
# 4.2 Dane sprzedażowe - Features

In [ ]:
# liczba nadań
vol_df = pd.read_parquet('data/Posting_Volumes')
vol_df['postingDateFk'] = pd.to_datetime(vol_df['postingDateFk'], format='%Y%m%d')
vol_df.set_index('postingDateFk', inplace=True)
vol_df['Day'] = vol_df.index.day
vol_df['Month'] = vol_df.index.month
vol_df['Day_of_year'] = vol_df.index.dayofyear
week_of_month = (vol_df.index.day - 1) // 7 + 1
vol_df['Week_of_month'] = week_of_month
vol_df['Week_of_year'] = vol_df.index.isocalendar().week

In [ ]:
vol_df.head()

In [ ]:
grouped_products = vol_df.pivot_table(values='Volume', index='postingDateFk', columns='Product', aggfunc='sum')
grouped_products = grouped_products.rename(columns={'APM': 'APM_sum', 'COURIER': 'COURIER_sum'})
grouped_products = lag_features(grouped_products, 'APM_sum', lag_steps=15)
grouped_products = lag_features(grouped_products, 'COURIER_sum', lag_steps=15)
grouped_products = grouped_products.drop(['APM_sum','COURIER_sum'], axis=1)
grouped_products

In [ ]:
grouped_products2 = vol_df.pivot_table(values='Volume', index='Week_of_year', columns='Product', aggfunc='mean')
grouped_products2 = grouped_products2.rename(columns={'APM': 'APM_w_y_avg', 'COURIER': 'COURIER_w_y_avg'})
grouped_products2.head(3)

In [ ]:
grouped_vol_d = vol_df.pivot_table(values='Volume', index='Day_of_year', aggfunc='mean')
grouped_vol_d = grouped_vol_d.rename(columns={'Volume': 'volume_d_y_avg'})
grouped_vol_d.tail()

In [ ]:
grouped_vol_w = vol_df.pivot_table(values='Volume', index='Week_of_year', aggfunc='mean')
grouped_vol_w = grouped_vol_w.rename(columns={'Volume': 'volume_w_y_avg'})
grouped_vol_w.head()

In [ ]:
grouped_vol_w2 = vol_df.pivot_table(values='Volume', index='Week_of_month', aggfunc='mean')
grouped_vol_w2 = grouped_vol_w2.rename(columns={'Volume': 'volume_w_m_avg'})
grouped_vol_w2.head()

In [ ]:
grouped_cust = vol_df.pivot_table(values='Volume', index='Week_of_year', columns='Customer', aggfunc='mean')
grouped_cust = grouped_cust.rename(columns={'Rest': 'Rest_volume_w_avg', 'X': 'X_volume_w_avg'})
grouped_cust.head(3)

In [ ]:
grouped_time = vol_df.groupby(['Month'])['Volume'].mean().reset_index()
grouped_time = grouped_time.rename(columns={'Volume': 'volume_m_avg'})
grouped_time.head(3)

<a id="FE_calendar"></a>
# 4.3 Dane kalendarzowe - Features

In [ ]:
time_data = pd.read_csv("data/dimDates_Task.csv", sep=";", index_col=['dateId'], parse_dates=['dateId'])
time_data = time_data[['dateQuarter','dateIsWeekend','dateIsHolidayInd','dateWeekDayStartsMonday', 
           'dateWeekOfYearStartsMonday','isThuHolyday','isFriHolyday', 'isSatHolyday', 'isWeekendParcelServiceBreakInd']]
time_data.index.max()

In [ ]:
time_data.head()

<a id="FE_pogodowe"></a>
# 4.4 Dane pogodowe - Features

In [ ]:
pogoda_df['Data'] = pd.to_datetime(dict(year=pogoda_df.Rok, month=pogoda_df.Miesiac, day=pogoda_df.Dzien))
pogoda_df['Week_of_year'] = pogoda_df['Data'].dt.isocalendar().week
pogoda_df.head(2)

In [ ]:
pogoda_agg_weekly = pogoda_df.groupby(['Week_of_year']).agg(
                    dobowa_temp_w_max = pd.NamedAgg(column='Maksymalna temperatura dobowa', aggfunc='max'),
                    dobowa_temp_w_min = pd.NamedAgg(column='Minimalna temperatura dobowa', aggfunc='min'),
                    dobowa_temp_w_avg = pd.NamedAgg(column='Srednia temperatura dobowa', aggfunc='mean'),
                    opady_w_avg = pd.NamedAgg(column='Suma dobowa opadow [mm]', aggfunc='mean'),
                    snieg_w_avg = pd.NamedAgg(column='Pokrywa sniegu [cm]', aggfunc='mean'),
                    ).reset_index()
pogoda_agg_weekly['temp_diff_avg'] = pogoda_agg_weekly['dobowa_temp_w_max'] - pogoda_agg_weekly['dobowa_temp_w_min']
pogoda_agg_weekly.head()

<a id="model_fin"></a>
# 5. XGBoost

In [ ]:
abt = pd.read_parquet('data/Posting_Volumes')
abt['postingDateFk'] = pd.to_datetime(abt['postingDateFk'], format='%Y%m%d')
abt.set_index('postingDateFk', inplace=True)
# ze względu na zmienne opóźnione
abt = abt.query("postingDateFk >= '2023-01-20'")
abt = abt.query("Customer == 'X'")
abt = abt[['Volume']].groupby('postingDateFk').sum()
# kolumny kalendarzowe
abt['Day'] = abt.index.day
abt['Month'] = abt.index.month
abt['Day_of_year'] = abt.index.dayofyear
week_of_month = (abt.index.day - 1) // 7 + 1
abt['Week_of_month'] = week_of_month
abt['Week_of_year'] = abt.index.isocalendar().week
end_train = '2023-06-30 00:00:00'
end_train_dt = pd.to_datetime(end_train)

In [ ]:
abt = abt.merge(orders_df, how='left', left_index=True, right_on='DateId')\
    .merge(grouped_products, how='left', left_index=True, right_on='postingDateFk')\
    .merge(grouped_products, how='left', left_index=True, right_on='postingDateFk')\
    .merge(time_data, how='left', left_index=True, right_index=True)

abt['copy_index'] = abt.index
abt = abt.merge(grouped_products2, how= 'left', on= 'Week_of_year')\
        .merge(grouped_vol_d, how= 'left', on= 'Day_of_year')\
        .merge(grouped_vol_w, how= 'left', on= 'Week_of_year')\
        .merge(grouped_vol_w2, how= 'left', on= 'Week_of_month')\
        .merge(grouped_cust, how= 'left', on= 'Week_of_year')\
        .merge(grouped_time, how= 'left', on= 'Month')\
        .merge(pogoda_agg_weekly, how= 'left', on= 'Week_of_year')


abt.set_index('copy_index', inplace=True)

# predykcja na tydzień przed więc bez opóźnien od 1 do 7
columns_to_drop = [col for col in abt.columns if 'lag_' in col and col.split('_')[1] not in ['1', '2', '3', '4', '5', '6', '7']]
abt = abt.drop(columns=columns_to_drop)
abt.head()

In [ ]:
X_train = abt.loc[:end_train].drop('Volume', axis=1)
X_test = abt.loc[~abt.index.isin(X_train.index)].drop('Volume', axis=1)
y_train = abt.loc[:end_train][['Volume']]
y_test = abt.loc[~abt.index.isin(y_train.index)][['Volume']]

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

param_grid = {

    'learning_rate': [0.01, 0.1, 0.2],

    'max_depth': [3, 5, 7],

    'subsample': [0.6, 0.8, 0.9]

}

grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=3)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

In [ ]:
# xgb_model = XGBRegressor(**best_params)
xgb_model = XGBRegressor()

xgb_model.fit(X_train, y_train)

In [ ]:
pred = xgb_model.predict(X_test)

In [ ]:
error_xg = mean_absolute_error(y_test, pred)
print(f"Error (MAE) Gradient Boosting: {error_xg:.2f}")

In [ ]:
pred_index = y_test.index

predictions_xgb = pd.Series(pred, index=pred_index)

fig, ax = plt.subplots(figsize=(9, 6), sharex=True, sharey=True)
abt.loc[:end_train].Volume.plot(ax=ax, label='train')
abt.loc[end_train:].Volume.plot(ax=ax, label='test')
predictions_xgb.plot(ax=ax, label='Xgboost')
ax.set_title('Model Xgboost - predykcja')
ax.set_xlabel('')
ax.legend()
plt.show()

In [ ]:
mse_xgboost = mean_squared_error(y_test.Volume, pred)
mape_xgboost = mean_absolute_percentage_error(y_test.Volume, pred)
mae_xgboost = mean_absolute_error(y_test.Volume, pred)

print("Metryka MSE:", mse_xgboost)
print("Metryka MAPE:", mape_xgboost)
print("Metryka MAE:", mae_xgboost)

<a id="comparison"></a>
# 6. Zestawienie wyników

In [ ]:
# SARIMA

print("Metryka MSE:", mse_sarima)
print("Metryka MAPE:", mape_sarima)
print("Metryka MAE:", mae_sarima)

In [ ]:
# XGBoost autoregresyjny

print("Metryka MSE:", mse_xgb_autoreg)
print("Metryka MAPE:", mape_xgb_autoreg)
print("Metryka MAE:", mae_xgb_autoreg)

In [ ]:
# Random Forest autoregresyjny

print("Metryka MSE:", mse_rf_autoreg)
print("Metryka MAPE:", mape_rf_autoreg)
print("Metryka MAE:", mae_rf_autoreg)

In [ ]:
# XGBoost

print("Metryka MSE:", mse_xgboost)
print("Metryka MAPE:", mape_xgboost)
print("Metryka MAE:", mae_xgboost)

Najniższe wartości błędów dla algorytmu XGBoost

<a id="poprawa"></a>
# 7. Ulepszenie rozwiązania

- zmienne opóźnione (lag), trendy opisujące zmiany w czasie oraz średnie z poprzednich lat dla danego tygodnia/miesiąca zostały wykorzystane jedynie na części tabel. Najlepszym rozwiązaniem byłoby sprawdzenie statystyk również na pozostałych tabelach

- sprawdzenie pozostałych algorytmów (random forest, catboost, lightgbm)

- wsparcie walidacji krzyżowej algorytmem bayesowskim i adaptacyjnym przeszukaniem przestrzeni hiperparametrów

- dokładniejsze przyjrzenie się zmiennym: feature importance i sprawdzenie, który rodzaj zmiennych jest najlepszym predyktorem i dobudowanie kolejnych zmiennych o podobnym charakterze

- sprawdzenie dni na których model myli się najbardziej, zbadanie rozkładu zmiennych i dobudowanie zmiennych, które mogą pomóc w lepszym opisaniu źle oszacowanych obserwacji

- uprodukcyjnienie rozwiązania: odpowiedź na pytanie, ktore zmienne możemy uwzględnić w tygodniowej/ miesięcznej predykcji (np. jeśli predykcja jest tygodniowa to najmniejsze zastosowanie opóźnienia zmiennych zależnych może wynieść 8)

- wydłużenie horyzontu czasowego modelu

- sprawdzenie innych modeli analizujących dane sekwencyjne: LSTM, temporal fusion transformer

- dodanie transformacij zmiennych opisujących sezonowość/ amplitudę np (Fourier transformation)

- odpowiednia transformacja wykorzystanych zmiennych - sprawdzenie jak sprawdzi się sinus/cosinus zmiennych typu dzień tygodnia

- sprawdzenie obserwacji odstających w szeregu czasowym, sprawdzenie jak na jakość predyckji wpłynie podstawienie pod anomalie np średniej z dwóch okresów

- końcowy model przewiduje dzienną liczbę nadań na podstawie zgłoszonego zapotrzebowania klienta 'X' oraz dodatkowych czynników wpływających na liczbę zamówień takich jak pogoda, dni świąteczne, dzień tygodnia, pora roku oraz liczba zamówień w poprzednich okresach. Należałoby wykonać analizę jak dany model sprawdza się na wszystkich klientach

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

In [ ]:
abt = abt.reset_index()
abt["time_idx"] = abt["copy_index"].dt.year + abt["copy_index"].dt.month + abt["copy_index"].dt.day
abt["client"] = 'X'

In [ ]:
abt_train = abt.loc[:end_train]
lista_kolumn = abt_train.columns.tolist()
lista_kolumn.remove("Volume")
lista_kolumn.remove("client")
lista_kolumn.remove("time_idx")

In [ ]:
abt_test = abt.loc[~abt.index.isin(X_train.index)]

In [ ]:
max_prediction_length = 7
max_encoder_length = 60

training = TimeSeriesDataSet(
    abt_train,
    time_idx="time_idx",
    target="Volume",
    group_ids= ['client'],
    #group_ids=["consumer_id"],
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    #static_categoricals=["consumer_id"],
    time_varying_known_reals=lista_kolumn,
    #time_varying_unknown_reals=['power_usage'],
    #target_normalizer=GroupNormalizer(
    #    groups=["consumer_id"], transformation="softplus"),  # we normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)


In [ ]:
abt_test = abt_test.fillna(0)

validation = TimeSeriesDataSet(
    abt_test,
    time_idx="time_idx",
    target="Volume",
    group_ids= ['client'],
    #group_ids=["consumer_id"],
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    #static_categoricals=["consumer_id"],
    time_varying_known_reals=lista_kolumn,
    #time_varying_unknown_reals=['power_usage'],
    #target_normalizer=GroupNormalizer(
    #    groups=["consumer_id"], transformation="softplus"),  # we normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [ ]:
batch_size = 300
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
import torch
from pytorch_forecasting import Baseline

actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [ ]:
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import lightning.pytorch as pl
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=True, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs")  

trainer = pl.Trainer(
    max_epochs=45,
    accelerator='cpu', 
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=160,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=160,
    output_size=7,  # there are 7 quantiles by default: [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
    loss=QuantileLoss(),
    log_interval=10, 
    reduce_on_plateau_patience=4)

In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)